In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import gensim
from sklearn.model_selection import train_test_split

import re
import string
from nltk.corpus import stopwords
import pkg_resources
import pickle
import nltk
import re, string, json
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [3]:
train = pd.read_csv('drive/MyDrive/Dataset/train.csv')
train = pd.DataFrame(train)

train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
train.shape

(27481, 4)

In [5]:
train = train[train['text'].str.split().str.len().lt(100)]

In [6]:
train.isnull().sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [7]:
train['sentiment'].value_counts()

neutral     11117
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [8]:
train.shape

(27480, 4)

In [9]:
def tokenize_twitter(sentences):
    """
    Tokenize sentences into tokens (words)
    
    Args:
        sentences: List of strings
    
    Returns:
        List of lists of tokens
    """
    print("Starting Cleaning Process")
    tokenized_sentences = []
    for sentence in tqdm(sentences):
        
        # Convert to lowercase letters
        sentence = cleanhtml(sentence)
        sentence = sentence.lower()
        sentence = _replace_urls(sentence)
        sentence = remove_email(sentence)
        sentence = misc(sentence)
        sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)

        tokenized = nltk.word_tokenize(sentence)
        
        # append the list of words to the list of lists
        tokenized_sentences.append(tokenized)
    
    return tokenized_sentences

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def _replace_urls(data):
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)
    return data

def remove_email(data):
    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)
    return data

def misc(data):
    # Remove new line characters
    data = re.sub('\s+', ' ', data)
    # Remove distracting single quotes
    data = re.sub("\'", "", data)
    return data

In [10]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("was"))

def lemma(sentences):
    print("Starting Lemmatization and Tokenizing Process")
    final = []
    for sentence in tqdm(sentences):
        output = [w.lower() for w in sentence]
        lemmatized_output = [lemmatizer.lemmatize(w) for w in output]
        final.append(lemmatized_output)
    
    return final

wa


In [12]:
def littleCleaning(sentences):
    print("Starting cleaning Process")
    ret_list = []
    for sentence in tqdm(sentences):
      for words in sentence:
        if len(words) == 1 and (words == 'a' or words == 'i'):
          pass
        elif len(words) > 1:
          pass
        else:
          sentence.remove(words)

      word_list = " ".join(sentence).strip(" ")
      ret_list.append(word_list)
    return ret_list

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
pro_sentences = []
#Splitting pd.Series to list
data_to_list = train['text'].values.tolist()

def normalization_pipeline(sentences):
    print("Starting Normalization Process")
    sentences = tokenize_twitter(sentences)
    sentences = lemma(sentences)
    sentences = littleCleaning(sentences)
    print("Normalization Process Finished")
    return sentences

pro_sentences = normalization_pipeline(data_to_list)
pro_sentences[: 5]

Starting Normalization Process
Starting Cleaning Process



Starting Lemmatization and Tokenizing Process



Starting cleaning Process



Normalization Process Finished


['i have responded if i were going',
 'sooo sad i will miss you here in san diego',
 'my bos is bullying me',
 'what interview leave me alone',
 'son of why couldn they put them on the release we already bought']

In [15]:
# fina_sentences = np.array(data_to_list)
fina_sentences = np.array(pro_sentences)

In [16]:
vocab = set()
for sentence in pro_sentences:
  for word in sentence.split(" "):
    vocab.add(word)

print(len(vocab))

21672


In [19]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
max_words = 10000 # 21672
max_len = 50

In [20]:
# Converting the sentiment labels from Neutral, Negative and Positive to a float type.

labels = np.array(train['sentiment'])
y = []
for i in range(len(labels)):
    if labels[i] == 'negative':
        y.append(-1)
    if labels[i] == 'positive':
        y.append(1)
    if labels[i] == 'neutral':
        y.append(0)
        
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")

In [21]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(fina_sentences)
sequences = tokenizer.texts_to_sequences(fina_sentences)
sequences[0]

[1, 17, 9209, 69, 1, 146, 45]

In [28]:
import pickle

with open('drive/MyDrive/Dataset/Models/tokenizerSequencing.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
data = pad_sequences(sequences, maxlen=max_len)

In [23]:
from sklearn.model_selection import train_test_split

#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state=0)

print (len(X_train),len(X_test),len(y_train),len(y_test))

20610 6870 20610 6870


In [24]:
# Bidirectional LTSM model

model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2.add(layers.Dense(3,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto',
                              period=1,save_weights_only=False)

history2 = model2.fit(X_train, y_train, epochs = 10,validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/10
645/645 [==============================] - 36s 25ms/step - loss: 1.0223 - accuracy: 0.4619 - val_loss: 0.8326 - val_accuracy: 0.6202

Epoch 00001: val_accuracy improved from -inf to 0.62023, saving model to best_model2.hdf5
Epoch 2/10
645/645 [==============================] - 15s 23ms/step - loss: 0.7800 - accuracy: 0.6567 - val_loss: 0.7455 - val_accuracy: 0.6726

Epoch 00002: val_accuracy improved from 0.62023 to 0.67263, saving model to best_model2.hdf5
Epoch 3/10
645/645 [==============================] - 15s 23ms/step - loss: 0.7054 - accuracy: 0.6985 - val_loss: 0.7289 - val_accuracy: 0.6958

Epoch 00003: val_accuracy improved from 0.67263 to 0.69578, saving model to best_model2.hdf5
Epoch 4/10
645/645 [==============================] - 15s 23ms/step - loss: 0.6618 - accuracy: 0.7267 - val_loss: 0.6929 - val_accuracy: 0.7058

Epoch 00004: val_accuracy improved from 0.69578 to 0.70582, saving model to best_model2.hdf5
Epoch 5/10
645/645 [==============================]

In [27]:
# Testing the model - array[1] is postive
# I just want to go home, not toil here and kiss her ass all day.
temp_sen = ["it is so hot today, but it is perfect for a beach party"]
processed = normalization_pipeline(temp_sen)
temp_seq = tokenizer.texts_to_sequences(processed)
temp_seq = pad_sequences(temp_seq, maxlen=max_len)
print(temp_seq)
model2.predict_proba(temp_seq)

Starting Normalization Process
Starting Cleaning Process



Starting Lemmatization and Tokenizing Process



Starting cleaning Process



Normalization Process Finished
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   5   9  16 265  43  19   5   9 525  11   4 543 291]]


/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.46517026, 0.44373482, 0.0910949 ]], dtype=float32)

In [25]:
labels[4:7]

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [26]:
y[4:7]

array([-1,  0,  1])

In [29]:
model2.save('drive/MyDrive/Dataset/Models/7ProjectModel.h5')

In [ ]:
"""
from tensorflow import keras
model = keras.models.load_model('path/to/location')
"""